In [ ]:
##PROCESSING OF 16S AMPLICON SEQUENCES##

In [ ]:
#Quality check
for N in `ls zr*_R1.fastq`; do fastx_trimmer -f 10 -l 280 -z -i $N -o $N.gz; done
for N in `ls zr*_R2.fastq`; do fastx_trimmer -f 7 -l 253 -z -i $N -o $N.gz; done

for N in `ls p*_R1.fastq`; do fastx_trimmer -f 7 -z -i $N -o $N.gz; done
for N in `ls p*_R2.fastq`; do fastx_trimmer -f 10 -l 240 -z -i $N -o $N.gz; done

for N in `ls a*_R1.fastq`; do fastx_trimmer -f 7 -z -i $N -o $N.gz; done
for N in `ls a*_R2.fastq`; do fastx_trimmer -f 10 -l 240 -z -i $N -o $N.gz; done

In [ ]:
#A list of files to assembly is generated.
ls *.fastq.gz | perl -pe 's/_R.*.fastq.gz//g' | sort | uniq >lista

#Assembly using CASPER. #Run assembly_CASPER.sh script to generate works
bash assembly_CASPER.sh mp_ensamble

#!/bin/bash
#$ -cwd
#$ -j y
#$ -S /bin/bash

SEQS=$(pwd)
SALIDAS=$(pwd)
BIN=/usr/local/bin
BIN2=/usr/local/bin
BIN3=/usr/bin
COUNT=0
for FAA in `cat lista`
do
let COUNT=COUNT+1
echo "#!/bin/bash" >$*.$COUNT.scr
echo "#$ -cwd" >>$*.$COUNT.scr
echo "#$ -j y" >>$*.$COUNT.scr
echo "#$ -S /bin/bash" >>$*.$COUNT.scr
echo "$BIN/casper -d 12 -k11 -w 12 -g 0.6 <(zcat $SEQS/$FAA"_R1.fastq.gz") <(zcat $SEQS/$FAA"_R2.fastq.gz") -o $FAA"_assembly"" >>$*.$COUNT.scr
echo "$BIN3/fastq_to_fasta -n  -i $FAA_assembly.fastq -o $FAA"_assembly.fas"" >>$*.$COUNT.scr
done


#Running works in the cluster
for N in `ls *.scr`; do qsub $N; done

In [ ]:
#Verifying sequence length for V3 region trimming
~/anaconda3/bin/seqkit stats p*_assembly.fas 
~/anaconda3/bin/seqkit stats p*_assembly.fas 

In [ ]:
#Removing the first 130 bases (V3 region) from the pulque samples 
for N in `ls p*_assembly.fas | sed 's/.fas//g'`; do fastx_trimmer -f 131 -i $N.fas -o $N.fasta; done
for N in `ls a*_assembly.fas | sed 's/.fas//g'`; do fastx_trimmer -f 131 -i $N.fas -o $N.fasta; done

In [ ]:
#Verifying sequence length for pulque sequences
~/anaconda3/bin/seqkit stats p*_assembly.fasta 
~/anaconda3/bin/seqkit stats p*_assembly.fasta 
#Renaming 
rm p*.fas a*.fas
for N in `ls *.fas | sed 's/.fas//g'`; do mv $N.fas $N.fasta; done

In [ ]:
#Number of merged sequences per sample
grep -c "@M" *_assembly.fastq > total_sec_ensambladas

In [ ]:
#Sequences names are changed with “header.fasta.numbers.pl”. The script is runned for each sample
for N in `ls *.fasta | sed 's/_assembly.fasta//g'`; do perl /home/angelica/scripts/header.fasta.numbers.pl $N $N"_assembly.fasta"; done
#Concatenate all samples
cat *.numbered.fas > mp_16S.fas
#Edit sequence name to leave only the first part, which refers to sample name.
perl -i.bak -pe "s/\ .*//g" mp_16S.fas
#Edit the name header
sed -i 's/_/./' mp_16S.fas
#Sequence clustering of OTUs at 97%  of identity was done with cd-hit-est, -c indicates the identity for clustering.
cd-hit-est -i mp_16S.fas -c 0.97 -o mp_16S.fasout -T 20 -M 0
#The clustering file is converted in a file that can be read by QIIME.
perl -pne 's/\t//g;s/^.*,//g;s/\.\.\..*$//g;s/\n/\t/g;s/\>Cluster\ /\n/g;s/\>//g; eof && do{chomp; print "$_ \n"; exit}' mp_16S.fasout.clstr > mp_16S.otu
sed -i '1d' mp_16S.otu
#Extract representative OTUs, -i indicates input file, -f is the fasta file with the sequences to be extracted. 
/home/qiime/bin/pick_rep_set.py -i mp_16S.otu -f mp_16S.fas -o mp_16S.rep.fna

In [ ]:
#Removing singletons
wc -l mp_16S.otu #236356
awk 'NF < 3' mp_16S.otu | cut -f1 >ids_REMOVE_singletons.txt
wc -l ids_REMOVE_singletons.txt #213787 ids_REMOVE_singletons.txt
cut -f1 mp_16S.otu > rawids

#Generating file with singleton's identities
awk 'NR==FNR { values[$0]; next } !($0 in values)' ids_REMOVE_singletons.txt rawids  >repsetWOsingletons.txt


#Extracting fasta without singletons 
perl -ne 'if(/^>(\S+)/){$c=$i{$1}}$c?print:chomp;$i{$_}=1 if @ARGV'repsetWOsingletons.txt mp_16S.rep.fna >lda_nosingletons.fna
grep -c ">" lda_nosingletons.fna #22569


#Removing contaminants (no 16S sequences)
parallel_assign_taxonomy_blast.py -i lda_nosingletons.fna -o ldaNO16S_screen1 -r /qiime/gg_otus-13_8-release/rep_set/70_otus.fasta -t /qiime/gg_otus-13_8-release/taxonomy/70_otu_taxonomy.txt -X LDA1

#Identifying sequences with no hits on Greengenes db at 70% identity
cat ldaNO16S_screen1/lda_nosingletons_tax_assignments.txt | grep "No blast hit" | cut -f1 >idsnohit.txt
wc -l idsnohit.txt #2194 idsnohit.txt

#Extracting sequences with no hit from fasta
perl -ne 'if(/^>(\S+)/){$c=$i{$1}}$c?print:chomp;$i{$_}=1 if @ARGV' idsnohit.txt lda_nosingletons.fna  >nohitGG.fasta

#Verifying that the sequences don't have a hit in the last version of Silva db
parallel_assign_taxonomy_blast.py -i nohitGG.fasta -o LDAsinhitGG -r /databases/silva/qiime/dada_silva138.fna -t /databases/silva/qiime/dada_silva138.tax.txt


#Obtaining sequence ids without singletons and contaminants (screened 16S)
grep -v "No blast hit" ldaNO16S_screen1/lda_nosingletons_tax_assignments.txt | cut -f1 >ggret1.txt
grep -v "No blast hit" LDAsinhitGG/nohitGG_tax_assignments.txt | cut -f1 >ggret2.txt
cat ggret*.txt >ret_screened_nosingleton.txt

wc -l ret_screened_nosingleton.txt #22251 ret_screened_nosingleton.txt

#Extracting sequences with no hit
perl -ne 'if(/^>(\S+)/){$c=$i{$1}}$c?print:chomp;$i{$_}=1 if @ARGV' ret_screened_nosingleton.txt  lda_nosingletons.fna >nosingleton_screened.fna

#Aligning to identify chimeric sequences
parallel_align_seqs_pynast.py -i nosingleton_screened.fna -o chimera_align -O 400 -X chimx

#Searching chimeras
parallel_identify_chimeric_seqs.py -m blast_fragments -i nosingleton_screened.fna -a chimera_align/nosingleton_screened_aligned.fasta -o singleton_chimera.txt -X chimerablast --id_to_taxonomy_fp /qiime/gg_otus-13_8-release/taxonomy/97_otu_taxonomy.txt -r /qiime/gg_otus-13_8-release/rep_set/97_otus.fasta -O 300
wc -l singleton_chimera.txt #5289 singleton_chimera.txt

#Obtaining chimeras ids
awk '{print $1}' singleton_chimera.txt >chimeraids

#Cleaning files
awk 'NR==FNR { values[$0]; next } !($0 in values)' chimeraids ret_screened_nosingleton.txt >ids_nochimera_screened_nosingleton.txt

wc -l  ids_nochimera_screened_nosingleton.txt #16962 ids_nochimera_screened_nosingleton.txt

#Creating OTU table without chimeras, filtered, and without singletons
awk 'NR==FNR { values[$0]; next } $1 in values { print }' ids_nochimera_screened_nosingleton.txt mp_16S.otu >nochimera_screened_singleton.otu

#Creating new fasta file withou chimeras, filtered, and iwthout singletons
perl -ne 'if(/^>(\S+)/){$c=$i{$1}}$c?print:chomp;$i{$_}=1 if @ARGV' ids_nochimera_screened_nosingleton.txt nosingleton_screened.fna >nochimera_screened_nosingleton.fna
grep -c ">" nochimera_screened_nosingleton.fna #16962


#Asigning taxonmy using Silva db
parallel_assign_taxonomy_blast.py -i nochimera_screened_nosingleton.fna -o tax -r /databases/silva/qiime/dada_silva138.fna -t /databases/silva/qiime/dada_silva138.tax.txt -O 300 -X tom

#Cleaning taxonomy
awk '{ for(i=1; i<=NF-2; i++) printf "%s%s", $i, (i<NF-2) ? OFS : ORS }' tax/nochimera_screened_nosingleton_tax_assignments.txt | perl -pe 's/\ /\_/g;s/_k/\tk/;s/\;_/\t/g' >tax_clean.txt

#Removing mithocondria and chloroplasts
grep -i 'chloroplast' tax_clean.txt | cut -f1 >tmprm1
grep -i 'mitochondria' tax_clean.txt | cut -f1 >>tmprm1

#Creating new OTU table without chimeras, filtered, without singletons, mitochondria and chloroplasts
awk 'NR==FNR { values[$0]; next } !($0 in values)' tmprm1 ids_nochimera_screened_nosingleton.txt >nochimera_screened_singleton_cpmt.ids
awk 'NR==FNR { values[$0]; next } $1 in values { print }' nochimera_screened_singleton_cpmt.ids nochimera_screened_singleton.otu >nochimera_screened_singleton_cpmt.otu
wc -l nochimera_screened_singleton_cpmt.otu #16888 nochimera_screened_singleton_cpmt.otu

#Creating OTU FASTA file without chimeras, filtered, without singletons, mitochondria and chloroplasts
perl -ne 'if(/^>(\S+)/){$c=$i{$1}}$c?print:chomp;$i{$_}=1 if @ARGV'  nochimera_screened_singleton_cpmt.ids nochimera_screened_nosingleton.fna >nochimera_screened_singleton_cpmt.fna

#Generating alignment with filtered OTUs
ssu-prep -f nochimera_screened_singleton_cpmt.fna mezcaltax 40 prefixcluster.txt
bash mezcaltax.ssu-align.sh
ssu-mask --stk2afa mezcaltax

#Cleaning alignment
perl -i.bak -pe 's/\./-/g;s/U/T/gi' mezcaltax/mezcaltax.bacteria.afa

# The generated file has 16840 sequences instead of 1692. The difference is due to ssu-align identified Archaea and removed
#Make phylogeny
./fasttreeMP -nt -fastest ssu-align/mezcaltax/mezcaltax.bacteria.afa >mezcal.nwk


#Creating OTU table in biom format
make_otu_table.py -i nochimera_screened_singleton_cpmt.otu -o nochimera_screened_singleton_cpmt.biom
biom convert --to-tsv -i nochimera_screened_singleton_cpmt.biom -o nochimera_screened_singleton_cpmt.biom.tsv
ln -s tax_clean.txt nochimera_screened_singleton_cpmt.tax

In [ ]:
#Phylogenetic tree 
cut -f1 mp_16S.biom.tsv>list_otu_ids
/home/qiime/bin/filter_fasta.py -f chimalign/mp_16S.screened_aligned.fasta -o mp_16S.biom.fna -s list_otu_ids 
qsub -pe completenode 4 -N arbolote -b y -j y -cwd -V " FastTreeMP -nt -gtr mp_16S.biom.fna > tree_mp_16S.nwk"